In [29]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
from IPython.display import display, Markdown
import tiktoken

load_dotenv(find_dotenv())

True

In [14]:
OpenAI_client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

In [40]:

def gpt_get_answer(instructions, prompt):
    completion = OpenAI_client.chat.completions.create(
      model="gpt-4o",
      messages=[
        {"role": "system", "content": instructions},
        {"role": "user", "content": prompt}
      ]
    )
    return completion.choices[0].message.content



In [17]:
def read_file(file_path):
    try:
        with open(file_path, 'r') as file:
            content = file.read()
        return content
    except FileNotFoundError:
        return "File not found."

file_path = 'prompt.txt'
file_content = read_file(file_path)

In [18]:
import os

def read_all_files_in_folder(folder_path):
    contents_list = []
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        if os.path.isfile(file_path):
            with open(file_path, 'r') as file:
                contents = file.read()
                contents_list.append(contents)
    return contents_list

folder_path = 'hook_examples'
all_contents = read_all_files_in_folder(folder_path)

# Print contents of all files
for idx, content in enumerate(all_contents):
    print(f"Contents of file {idx + 1}:\n{content}\n")

Contents of file 1:
// SPDX-License-Identifier: MIT
pragma solidity ^0.8.0;

import {BaseHook} from "v4-periphery/BaseHook.sol";
import {ERC20} from "solmate/tokens/ERC20.sol";

import {CurrencyLibrary, Currency} from "v4-core/types/Currency.sol";
import {PoolKey} from "v4-core/types/PoolKey.sol";
import {BalanceDeltaLibrary, BalanceDelta} from "v4-core/types/BalanceDelta.sol";

import {IPoolManager} from "v4-core/interfaces/IPoolManager.sol";

import {Hooks} from "v4-core/libraries/Hooks.sol";

contract PointsHook is BaseHook, ERC20 {
	// Use CurrencyLibrary and BalanceDeltaLibrary
	// to add some helper functions over the Currency and BalanceDelta
	// data types 
	using CurrencyLibrary for Currency;
    using BalanceDeltaLibrary for BalanceDelta;

	// Keeping track of user => referrer
	mapping(address => address) public referredBy;

	// Amount of points someone gets for referring someone else
    uint256 public constant POINTS_FOR_REFERRAL = 500 * 10 ** 18;

	// Initialize BaseHook a

3

In [25]:
final_prompt = file_content
for idx, content in enumerate(all_contents):
     final_prompt = final_prompt + f"""----------
                     HOOK EXAMPLE {idx}:
                     {content}


                     
                     """

In [27]:
len(final_prompt)

14137

In [31]:
enc = tiktoken.encoding_for_model("gpt-4o")

In [38]:
len(enc.encode(final_prompt))

3069

In [41]:
gpt_get_answer(final_prompt, "I want a simple hook that just has a counter that sums 1 every time a swap is done")

'To create a simple hook that increments a counter each time a swap is conducted, you can build on one of the examples provided. We\'ll create a basic hook contract that implements the `beforeSwap` hook method to achieve this. This hook will maintain a counter `_swapCount` and increment it every time the `beforeSwap` function is invoked.\n\nHere is a straightforward implementation:\n\n```solidity\n// SPDX-License-Identifier: MIT\npragma solidity ^0.8.24;\n\nimport {BaseHook} from "v4-periphery/BaseHook.sol";\n\nimport {Hooks} from "v4-core/src/libraries/Hooks.sol";\nimport {IPoolManager} from "v4-core/src/interfaces/IPoolManager.sol";\nimport {PoolKey} from "v4-core/src/types/PoolKey.sol";\nimport {PoolId, PoolIdLibrary} from "v4-core/src/types/PoolId.sol";\nimport {BeforeSwapDelta, BeforeSwapDeltaLibrary} from "v4-core/src/types/BeforeSwapDelta.sol";\n\ncontract SimpleSwapCounter is BaseHook {\n    using PoolIdLibrary for PoolKey;\n\n    // State variable to hold the swap count\n    u

In [42]:
aa = 'To create a simple hook that increments a counter each time a swap is conducted, you can build on one of the examples provided. We\'ll create a basic hook contract that implements the `beforeSwap` hook method to achieve this. This hook will maintain a counter `_swapCount` and increment it every time the `beforeSwap` function is invoked.\n\nHere is a straightforward implementation:\n\n```solidity\n// SPDX-License-Identifier: MIT\npragma solidity ^0.8.24;\n\nimport {BaseHook} from "v4-periphery/BaseHook.sol";\n\nimport {Hooks} from "v4-core/src/libraries/Hooks.sol";\nimport {IPoolManager} from "v4-core/src/interfaces/IPoolManager.sol";\nimport {PoolKey} from "v4-core/src/types/PoolKey.sol";\nimport {PoolId, PoolIdLibrary} from "v4-core/src/types/PoolId.sol";\nimport {BeforeSwapDelta, BeforeSwapDeltaLibrary} from "v4-core/src/types/BeforeSwapDelta.sol";\n\ncontract SimpleSwapCounter is BaseHook {\n    using PoolIdLibrary for PoolKey;\n\n    // State variable to hold the swap count\n    uint256 private _swapCount;\n\n    constructor(IPoolManager _poolManager) BaseHook(_poolManager) {}\n\n    function getHookPermissions() public pure override returns (Hooks.Permissions memory) {\n        return Hooks.Permissions({\n            beforeInitialize: false,\n            afterInitialize: false,\n            beforeAddLiquidity: false,\n            afterAddLiquidity: false,\n            beforeRemoveLiquidity: false,\n            afterRemoveLiquidity: false,\n            beforeSwap: true,\n            afterSwap: false,\n            beforeDonate: false,\n            afterDonate: false\n         });\n    }\n\n    // Getter function to retrieve the swap count\n    function getSwapCount() external view returns (uint256) {\n        return _swapCount;\n    }\n\n    function beforeSwap(address, PoolKey calldata, IPoolManager.SwapParams calldata, bytes calldata)\n        external\n        override\n        returns (bytes4, BeforeSwapDelta, uint24)\n    {\n        _swapCount++;\n        return (BaseHook.beforeSwap.selector, BeforeSwapDeltaLibrary.ZERO_DELTA, 0);\n    }\n}\n```\n\n### Explanation:\n- **State Variable (`_swapCount`)**: This variable keeps track of the number of swaps.\n- **Constructor**: Initializes the `BaseHook` contract with a reference to the `IPoolManager`.\n- **`getHookPermissions` Function**: Defines that this hook will utilize the `beforeSwap` function.\n- **Getter Function (`getSwapCount`)**: Allows external contracts or users to query the current swap count.\n- **`beforeSwap` Function**: Increments the `_swapCount` every time a swap occurs. The `BeforeSwapDeltaLibrary.ZERO_DELTA` and `0` returned indicate no changes to swap deltas or fees.\n\nThis contract can then be deployed and used within the Uniswap V4 infrastructure to keep track of swap counts for the associated pool.'

In [44]:
display(Markdown(aa))

To create a simple hook that increments a counter each time a swap is conducted, you can build on one of the examples provided. We'll create a basic hook contract that implements the `beforeSwap` hook method to achieve this. This hook will maintain a counter `_swapCount` and increment it every time the `beforeSwap` function is invoked.

Here is a straightforward implementation:

```solidity
// SPDX-License-Identifier: MIT
pragma solidity ^0.8.24;

import {BaseHook} from "v4-periphery/BaseHook.sol";

import {Hooks} from "v4-core/src/libraries/Hooks.sol";
import {IPoolManager} from "v4-core/src/interfaces/IPoolManager.sol";
import {PoolKey} from "v4-core/src/types/PoolKey.sol";
import {PoolId, PoolIdLibrary} from "v4-core/src/types/PoolId.sol";
import {BeforeSwapDelta, BeforeSwapDeltaLibrary} from "v4-core/src/types/BeforeSwapDelta.sol";

contract SimpleSwapCounter is BaseHook {
    using PoolIdLibrary for PoolKey;

    // State variable to hold the swap count
    uint256 private _swapCount;

    constructor(IPoolManager _poolManager) BaseHook(_poolManager) {}

    function getHookPermissions() public pure override returns (Hooks.Permissions memory) {
        return Hooks.Permissions({
            beforeInitialize: false,
            afterInitialize: false,
            beforeAddLiquidity: false,
            afterAddLiquidity: false,
            beforeRemoveLiquidity: false,
            afterRemoveLiquidity: false,
            beforeSwap: true,
            afterSwap: false,
            beforeDonate: false,
            afterDonate: false
         });
    }

    // Getter function to retrieve the swap count
    function getSwapCount() external view returns (uint256) {
        return _swapCount;
    }

    function beforeSwap(address, PoolKey calldata, IPoolManager.SwapParams calldata, bytes calldata)
        external
        override
        returns (bytes4, BeforeSwapDelta, uint24)
    {
        _swapCount++;
        return (BaseHook.beforeSwap.selector, BeforeSwapDeltaLibrary.ZERO_DELTA, 0);
    }
}
```

### Explanation:
- **State Variable (`_swapCount`)**: This variable keeps track of the number of swaps.
- **Constructor**: Initializes the `BaseHook` contract with a reference to the `IPoolManager`.
- **`getHookPermissions` Function**: Defines that this hook will utilize the `beforeSwap` function.
- **Getter Function (`getSwapCount`)**: Allows external contracts or users to query the current swap count.
- **`beforeSwap` Function**: Increments the `_swapCount` every time a swap occurs. The `BeforeSwapDeltaLibrary.ZERO_DELTA` and `0` returned indicate no changes to swap deltas or fees.

This contract can then be deployed and used within the Uniswap V4 infrastructure to keep track of swap counts for the associated pool.